# Load model from .pb file

This code will load and return the graph.

* `model_file_name`: path to a `.pb` file.
* `node_mapping`: mapping from the name of node in the graph to a Tensorflow node in current session

In [1]:
import tensorflow as tf
from tensorflow.python.platform import gfile
def load_graph(sess, model_file_name, node_mapping={}):
    with gfile.GFile(model_file_name, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, node_mapping)

This code will load the top-tagging model weights.

Please change the `model_file_name` to the file name of your model.

In [2]:
with tf.Session() as sess:
    # Create a placeholder for the input
    input_node = tf.placeholder(tf.float32, shape = [None, 224, 224, 3], name='Placeholder')
    graph_def = load_graph(sess=sess,
                           model_file_name='weights-floatingpoint-224x224-fixval-best/constantgraph.pb',
                           node_mapping={'Placeholder': input_node})
# Get the node for output
output_node = tf.get_default_graph().get_tensor_by_name("import/classifier/model_1/classifier_output/Softmax:0")

# Export to Tensorboard

This code will save the graph into a logging directory so that we can inspect it with Tensorboard.

* `log_dir_name`: Path to the directory that is used to store the log. You can use `tensorboard --logdir <log_dir_name>` to inspect the graph.

In [ ]:
def write_to_log(log_dir_name):
    LOGDIR='log-top-tagging-resnet50-2'
    train_writer = tf.summary.FileWriter(log_dir_name)
    train_writer.add_graph(sess.graph)

This code will save the top-tagging model to a folder.

Please change the parameter to where you want to store the Tensorboad information.

In [ ]:
write_to_log('log-top-tagging-resnet50')

# Run an inference

This code will run an inference on a matrix

In [4]:
import numpy as np
with tf.Session() as sess:
    y = sess.run(output_node, feed_dict={input_node: np.zeros((2, 224, 224, 3))})

In [5]:
print(y)

[[0.00170123 0.9982988 ]
 [0.00170123 0.9982988 ]]


# Evaluate model

Those are helper functions from [util.py](https://github.com/nhanvtran/MachineLearningNotebooks/blob/nvt/bwcustomweights-validate/project-brainwave/utils.py)

In [6]:
def normalize_and_rgb(images): 
    import numpy as np
    #normalize image to 0-255 per image.
    image_sum = 1/np.sum(np.sum(images,axis=1),axis=-1)
    given_axis = 0
    # Create an array which would be used to reshape 1D array, b to have 
    # singleton dimensions except for the given axis where we would put -1 
    # signifying to use the entire length of elements along that axis  
    dim_array = np.ones((1,images.ndim),int).ravel()
    dim_array[given_axis] = -1
    # Reshape b with dim_array and perform elementwise multiplication with 
    # broadcasting along the singleton dimensions for the final output
    image_sum_reshaped = image_sum.reshape(dim_array)
    images = images*image_sum_reshaped*255

    # make it rgb by duplicating 3 channels.
    images = np.stack([images, images, images],axis=-1)
    
    return images

In [11]:
def count_events(train_files):
    import tables
    n_events = 0
    for train_file in train_files:
        f = tables.open_file(train_file, 'r')
        n_events += f.root.label.shape[0]
        f.close()
    return n_events

In [18]:
def chunks(files, chunksize, max_q_size=4, shuffle=True): 
    """Yield successive n-sized chunks from a and b.""" 
    import tables
    import numpy as np
    for train_file in files: 
        f = tables.open_file(train_file, 'r') 
        nrows = f.root.label.nrows
        for istart in range(0,nrows,max_q_size*chunksize):  
            a = np.array(f.root.img_pt[istart:istart+max_q_size*chunksize]) # Images 
            b = np.array(f.root.label[istart:istart+max_q_size*chunksize]) # Labels 
            if shuffle: 
                c = np.c_[a.reshape(len(a), -1), b.reshape(len(b), -1)] # shuffle within queue size
                np.random.shuffle(c)
                test_images = c[:, :a.size//len(a)].reshape(a.shape)
                test_labels = c[:, a.size//len(a):].reshape(b.shape)
            else:
                test_images = a
                test_labels = b
            for jstart in range(0,len(test_labels),chunksize): 
                yield normalize_and_rgb(test_images[jstart:jstart+chunksize].copy()),test_labels[jstart:jstart+chunksize].copy(), len(test_labels[jstart:jstart+chunksize].copy())  
        f.close()

In [33]:
def test_model(preds, in_images, test_files, chunk_size=64, shuffle=True):
    """Test the model"""
    import tensorflow as tf
    from keras import backend as K
    from keras.objectives import binary_crossentropy 
    import numpy as np
    from keras.metrics import categorical_accuracy
    from tqdm import tqdm
    
    in_labels = tf.placeholder(tf.float32, shape=(None, 2))
    
    cross_entropy = tf.reduce_mean(binary_crossentropy(in_labels, preds))
    accuracy = tf.reduce_mean(categorical_accuracy(in_labels, preds))
    auc = tf.metrics.auc(tf.cast(in_labels, tf.bool), preds)
   
    n_test_events = count_events(test_files)
    chunk_num = int(n_test_events/chunk_size)+1
    preds_all = []
    label_all = []
    
    sess = tf.Session()
    sess.run(tf.local_variables_initializer())
    
    avg_accuracy = 0
    avg_auc = 0
    avg_test_loss = 0
    is_training = tf.get_default_graph().get_tensor_by_name('import/is_training:0')
    n_current_events = 0
    for img_chunk, label_chunk, real_chunk_size in chunks(test_files, chunk_size, shuffle=shuffle):
        test_loss, accuracy_result, auc_result, preds_result = sess.run([cross_entropy, accuracy, auc, preds],
                        feed_dict={in_images: img_chunk,
                                   in_labels: label_chunk,
                                   K.learning_phase(): 0,
                                   is_training: False})
        avg_test_loss += test_loss * real_chunk_size / n_test_events
        avg_accuracy += accuracy_result * real_chunk_size / n_test_events
        avg_auc += auc_result[0]  * real_chunk_size / n_test_events 
        preds_all.extend(preds_result)
        label_all.extend(label_chunk)
        n_current_events += real_chunk_size
    
        print("test_loss = ", "{:.3f}".format(avg_test_loss*n_test_events/n_current_events), end="")
        print("Test Accuracy:", "{:.3f}".format(avg_accuracy*n_test_events/n_current_events), ", Area under ROC curve:", "{:.3f}".format(avg_auc*n_test_events/n_current_events))
    
    return avg_test_loss, avg_accuracy, avg_auc, np.asarray(preds_all).reshape(n_test_events,2), np.asarray(label_all).reshape(n_test_events,2)

This will test the model on a test file.

Please change the parameter so that it uses the correct file.

In [34]:
test_model(output_node, input_node, ['img224_all/converted/rotation_224_v1/test_file_0.h5'])

test_loss =  0.229Test Accuracy: 0.922 , Area under ROC curve: 0.000
test_loss =  0.259Test Accuracy: 0.922 , Area under ROC curve: 0.484
test_loss =  0.259Test Accuracy: 0.922 , Area under ROC curve: 0.645
test_loss =  0.290Test Accuracy: 0.906 , Area under ROC curve: 0.726
test_loss =  0.276Test Accuracy: 0.906 , Area under ROC curve: 0.774
test_loss =  0.251Test Accuracy: 0.911 , Area under ROC curve: 0.807
test_loss =  0.245Test Accuracy: 0.915 , Area under ROC curve: 0.831
test_loss =  0.268Test Accuracy: 0.908 , Area under ROC curve: 0.849
test_loss =  0.252Test Accuracy: 0.911 , Area under ROC curve: 0.863
test_loss =  0.258Test Accuracy: 0.909 , Area under ROC curve: 0.874
test_loss =  0.264Test Accuracy: 0.908 , Area under ROC curve: 0.883
test_loss =  0.275Test Accuracy: 0.902 , Area under ROC curve: 0.891
test_loss =  0.276Test Accuracy: 0.900 , Area under ROC curve: 0.897
test_loss =  0.273Test Accuracy: 0.901 , Area under ROC curve: 0.902
test_loss =  0.283Test Accuracy: 0

KeyboardInterrupt: 

# Load model from .h5 file

Change `file_name` to the path of model.

In [2]:
import keras
file_name = '../weights-quantized-224x224-fixval-best-final/class_model_best.h5'
new_model = keras.models.load_model(file_name)
new_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 1, 2048)        0         
_________________________________________________________________
dense_1 (Dense)              (None, 1, 1, 1024)        2098176   
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
classifier_output (Dense)    (None, 2)                 2050      
Total params: 2,100,226
Trainable params: 2,100,226
Non-trainable params: 0
_________________________________________________________________
